# Probabilistic program synthesis for tabular data using CrossCat+CGPM

<img src="../resources/bayesian-program-synthesis.jpg"/>

$$
\mathcal{G} =
    \mathcal{G}^{\rm primitive} \; \lvert \;
    \mathcal{G}\times\mathcal{G} \; \lvert \;
    \mathcal{G} \stackrel{\rm chain}\to \mathcal{G} \; \lvert \;
    [\mathcal{G}\dots\mathcal{G}] \; \lvert \;
    \cup \mathcal{G}
$$

In [1]:
import numpy as np
import pandas as pd

In [2]:
prng = np.random.RandomState(10)

#### Prepare the population schema.

In [3]:
schema = [
    ["Country_of_Operator",          ['categorical', {'k': 79}]],  # 0
    ["Operator_Owner",               ['categorical', {'k': 346}]], # 1
    ["Users",                        ['categorical', {'k': 18}]],  # 2
    ["Purpose",                      ['categorical', {'k': 46}]],  # 3
    ["Class_of_Orbit",               ['categorical', {'k': 4}]],   # 4
    ["Type_of_Orbit",                ['categorical', {'k': 7}]],   # 5
    ["Perigee_km",                   ['normal', None]],            # 6
    ["Apogee_km",                    ['normal', None]],            # 7
    ["Eccentricity",                 ['normal', None]],            # 8
    ["Period_minutes",               ['normal', None]],            # 9
    ["Launch_Mass_kg",               ['normal', None]],            # 10
    ["Dry_Mass_kg",                  ['normal', None]],            # 11
    ["Power_watts",                  ['normal', None]],            # 12
    ["Date_of_Launch",               ['normal', None]],            # 13
    ["Anticipated_Lifetime",         ['normal', None]],            # 14
    ["Contractor",                   ['categorical', {'k': 282}]], # 15
    ["Country_of_Contractor",        ['categorical', {'k': 54}]],  # 16
    ["Launch_Site",                  ['categorical', {'k': 25}]],  # 17
    ["Launch_Vehicle",               ['categorical', {'k': 141}]], # 18
    ["Source_Used_for_Orbital_Data", ['categorical', {'k': 38}]],  # 19
    ["longitude_radians_of_geo",     ['normal', None]],            # 20
    ["Inclination_radians",          ['normal', None]],            # 21
]

#### Sample an AST from the prior.

In [4]:
from cgpm2 import sample_crosscat

In [5]:
ast = sample_crosscat.generate_random_ast(schema, prng)

In [6]:
ast

[[[100000, ('crp', None), {'alpha': 1.121268}],
  [[0, ['categorical', {'k': 79}], {'alpha': 0.719775}],
   [8,
    ['normal', None],
    {'m': 1.710845, 'nu': 0.433329, 'r': 0.221829, 's': 1.943864}],
   [11,
    ['normal', None],
    {'m': 0.163875, 'nu': 4.392115, 'r': 0.484694, 's': 2.88707}]]],
 [[100001, ('crp', None), {'alpha': 0.583097}],
  [[1, ['categorical', {'k': 346}], {'alpha': 1.050958}],
   [4, ['categorical', {'k': 4}], {'alpha': 0.737407}],
   [5, ['categorical', {'k': 7}], {'alpha': 2.393045}],
   [7,
    ['normal', None],
    {'m': 1.76423, 'nu': 0.793158, 'r': 0.048032, 's': 0.984268}],
   [12,
    ['normal', None],
    {'m': 0.609366, 'nu': 0.909741, 'r': 1.749406, 's': 0.289516}],
   [16, ['categorical', {'k': 54}], {'alpha': 0.04078}],
   [20,
    ['normal', None],
    {'m': 0.425179, 'nu': 1.440711, 'r': 0.044054, 's': 2.119557}],
   [21,
    ['normal', None],
    {'m': 2.104527, 'nu': 0.72045, 'r': 0.540442, 's': 0.930333}]]],
 [[100002, ('crp', None), {'alpha

#### Compile the AST into the "Core DSL".

In [7]:
core_dsl = sample_crosscat.compile_ast_to_core_dsl(ast)

In [8]:
print core_dsl.getvalue()

- view:
    row clustering model:
      - crp{id:100000}:
          distargs:
          hypers:
            alpha: 1.1213
    distribution models:
      - categorical{id:0}:
          distargs:
            k: 79
          hypers:
            alpha: 0.7198
      - normal{id:8}:
          distargs:
          hypers:
            s: 1.9439
            r: 0.2218
            m: 1.7108
            nu: 0.4333
      - normal{id:11}:
          distargs:
          hypers:
            s: 2.8871
            r: 0.4847
            m: 0.1639
            nu: 4.3921
- view:
    row clustering model:
      - crp{id:100001}:
          distargs:
          hypers:
            alpha: 0.5831
    distribution models:
      - categorical{id:1}:
          distargs:
            k: 346
          hypers:
            alpha: 1.0510
      - categorical{id:4}:
          distargs:
            k: 4
          hypers:
            alpha: 0.7374
      - categorical{id:5}:
          distargs:
            k: 7
          hypers

#### Compile "Core DSL" into the "Embedded DSL".

In [9]:
embedded_dsl = sample_crosscat.compile_core_dsl_to_embedded_dsl(core_dsl.getvalue())

In [10]:
print embedded_dsl.getvalue()

from cgpm2.categorical import Categorical
from cgpm2.crp import CRP
from cgpm2.flexible_rowmix import FlexibleRowMixture
from cgpm2.normal import Normal
from cgpm2.poisson import Poisson
from cgpm2.product import Product

nan = float('nan')

view0 = FlexibleRowMixture(
  cgpm_row_divide=CRP(outputs=[100000], inputs=[], hypers={'alpha': 1.1213},),
  cgpm_components_base=Product(cgpms=[
    Categorical(outputs=[0], inputs=[], distargs={'k': 79}, hypers={'alpha': 0.7198},),
    Normal(outputs=[8], inputs=[], hypers={'s': 1.9439, 'r': 0.2218, 'm': 1.7108, 'nu': 0.4333},),
    Normal(outputs=[11], inputs=[], hypers={'s': 2.8871, 'r': 0.4847, 'm': 0.1639, 'nu': 4.3921},),])
)
view1 = FlexibleRowMixture(
  cgpm_row_divide=CRP(outputs=[100001], inputs=[], hypers={'alpha': 0.5831},),
  cgpm_components_base=Product(cgpms=[
    Categorical(outputs=[1], inputs=[], distargs={'k': 346}, hypers={'alpha': 1.051},),
    Categorical(outputs=[4], inputs=[], distargs={'k': 4}, hypers={'alpha': 0.7374},),


#### Execute the Embeddd DSL source code to build the model trace.

In [11]:
exec(embedded_dsl.getvalue())

In [12]:
crosscat

#### Load observations for .csv file and observe them into the model trace.

In [13]:
df = pd.read_csv('../resources/satellites.coded.csv', index_col=False)
for rowid, values in df.iterrows():
    observation = dict(zip(range(len(values)), values.values))
    crosscat.observe(rowid, observation)

#### Run Bayesian synthesis.

In [14]:
from cgpm2.transition_crosscat import GibbsCrossCat
synthesizer = GibbsCrossCat(crosscat)
synthesizer.transition_structure_cpp(N=10)
synthesizer.transition(N=10, kernels=['hypers_distributions', 'hypers_row_divide'])

Completed: 10 iterations in 2.671450 seconds.
Completed: 10 iterations in 28.322833 seconds.


#### Render the posterior model trace as Embedded DSL + sequence of observes that reconstruct it exactly.

In [15]:
print sample_crosscat.render_trace_in_embedded_dsl(synthesizer.crosscat).getvalue()

from cgpm2.categorical import Categorical
from cgpm2.crp import CRP
from cgpm2.flexible_rowmix import FlexibleRowMixture
from cgpm2.normal import Normal
from cgpm2.poisson import Poisson
from cgpm2.product import Product

nan = float('nan')

view0 = FlexibleRowMixture(
  cgpm_row_divide=CRP(outputs=[385995556], inputs=[], hypers={'alpha': 2.0763},),
  cgpm_components_base=Product(cgpms=[
    Normal(outputs=[8], inputs=[], hypers={'s': 0.1953, 'r': 0.0009, 'm': 0.4119, 'nu': 1166.0},),])
)
view1 = FlexibleRowMixture(
  cgpm_row_divide=CRP(outputs=[1677575585], inputs=[], hypers={'alpha': 3.3791},),
  cgpm_components_base=Product(cgpms=[
    Categorical(outputs=[1], inputs=[], distargs={'k': 346}, hypers={'alpha': 0.2959},),
    Categorical(outputs=[4], inputs=[], distargs={'k': 4}, hypers={'alpha': 0.0159},),
    Normal(outputs=[7], inputs=[], hypers={'s': 9456749226.0409, 'r': 0.0009, 'm': 97557.9655, 'nu': 1166.0},),
    Normal(outputs=[21], inputs=[], hypers={'s': 12.0679, 'r': 0.002

#### Render the posterior model trace in VentureScript.

In [16]:
print sample_crosscat.render_trace_in_venturescript(synthesizer.crosscat, schema, observes=True).getvalue()

assume variable_to_distributions = dict(
    // Mixture models in view 0.
    ["Eccentricity",	[
        make_nig_normal(0.4119, 0.0009, 0.1953, 1166.0000),
        make_nig_normal(0.4119, 0.0009, 0.1953, 1166.0000),
        make_nig_normal(0.4119, 0.0009, 0.1953, 1166.0000),
        make_nig_normal(0.4119, 0.0009, 0.1953, 1166.0000),
        make_nig_normal(0.4119, 0.0009, 0.1953, 1166.0000),
        make_nig_normal(0.4119, 0.0009, 0.1953, 1166.0000),
        make_nig_normal(0.4119, 0.0009, 0.1953, 1166.0000),
        make_nig_normal(0.4119, 0.0009, 0.1953, 1166.0000),
        make_nig_normal(0.4119, 0.0009, 0.1953, 1166.0000),
        make_nig_normal(0.4119, 0.0009, 0.1953, 1166.0000),
        make_nig_normal(0.4119, 0.0009, 0.1953, 1166.0000),
        make_nig_normal(0.4119, 0.0009, 0.1953, 1166.0000),
        make_nig_normal(0.4119, 0.0009, 0.1953, 1166.0000)
        ]],
    // Mixture models in view 1.
    ["Operator_Owner",	[
        make_symm_dirichlet_categorical(0.2959),
     